In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sys
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import logit

from sepsis_utils import sepsis_utils as su
from sepsis_utils import roc_utils as ru

from sklearn.pipeline import Pipeline

# used for train/test splits
from sklearn.cross_validation import train_test_split

# used to impute mean for data
from sklearn.preprocessing import Imputer

# normalize the data
from sklearn import preprocessing

# logistic regression is our model of choice
from sklearn.linear_model import LogisticRegression

# used to create confusion matrix
from sklearn.metrics import confusion_matrix

from sklearn.cross_validation import cross_val_score

# used to calculate AUROC/accuracy
from sklearn import metrics

# for calibration curve of severity scores
from sklearn.calibration import calibration_curve

# default colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']
%matplotlib inline

from __future__ import print_function

In [ ]:
# create a database connection

# below config used on pc70
sqluser = 'alistairewj'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)

Each function extracts the data from a materialized view. See the *mimic-code* repository for instructions on how to create the materialized views.

In [ ]:
# call functions to extract the severity scores
qsofa = su.get_qsofa(con)
sofa = su.get_sofa(con)
oasis = su.get_oasis(con)
lods = su.get_lods(con)
sirs = su.get_sirs(con)
angus = su.get_angus(con)

# Time of suspected infection

Suspected infection is defined as:

* Antibiotics within 72 hours of a culture
* A culture within 24 hours of antibiotics

We can extract antibiotic usage from the, PRESCRIPTIONS, INPUTEVENTS_MV and INPUTEVENTS_CV tables. We can extract time of blood cultures from the MICROBIOLOGYEVENTS table. Detail is given in defining-suspected-infection.ipynb.

In [ ]:
ab = su.get_suspected_infection_time(con)

# Other data

This query extracts other data of interest:

* Age
* Gender
* Immunosuppression
* BMI
* Metastatic cancer (Elixhauser comorbidity)
* Diabetes (Elixhauser comorbidity)


In [ ]:
misc = su.get_other_data(con)
print('{} ICU stays.'.format(misc.shape[0]))
idx = misc.age > 1
print('{} adult ICU stays.'.format(np.sum(idx)))
demog_col = ['height','weight','bmi']
for c in demog_col:
    print('\t{:2.2f}% have {}.'.format( (np.sum(idx) - misc[c][idx].isnull().sum())*100.0 / np.sum(idx), c ))

# Cohort

The below code creates our cohort of interest. This cohort is used to apply inclusion criteria by means of an inner join. Inclusion criteria are:

* Adult patient, i.e. age >= 16
* First ICU stay for the patient

In [ ]:
cohort = su.get_cohort(con)

# close the database connection as we are finished extracting data
con.close()